# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [2]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
mean_fare = df.groupby(['Sex'])['Fare'].mean().reset_index()
mode_fare = df.groupby(['Sex'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
sex_count = df.groupby(['Sex'])['Fare'].agg({'sex_count':'size'}).reset_index()
temp = pd.merge(mean_fare, mode_fare, on=['Sex'], how='left')
temp = pd.merge(temp, sex_count, on=['Sex'], how='left')
temp.columns = ['Sex', 'mean_fare', 'mode_fare', 'sex_count']
df = pd.merge(df, temp, on=['Sex'], how='left')
df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,mean_fare,mode_fare,sex_count
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,25.523893,8.05,577
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,44.479818,7.75,314
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,44.479818,7.75,314
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44.479818,7.75,314
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,25.523893,8.05,577
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,25.523893,8.05,577
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,44.479818,7.75,314
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,44.479818,7.75,314
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,25.523893,8.05,577


In [4]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

8 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'mean_fare', 'mode_fare', 'sex_count']



,Pclass,Age,SibSp,Parch,Fare,mean_fare,mode_fare,sex_count
0,3,22.0,1,0,7.2500,25.523893,8.05,577
1,1,38.0,1,0,71.2833,44.479818,7.75,314
2,3,26.0,0,0,7.9250,44.479818,7.75,314
3,1,35.0,1,0,53.1000,44.479818,7.75,314
4,3,35.0,0,0,8.0500,25.523893,8.05,577


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

### Ans: 是

In [5]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_minus = df.drop(['mean_fare', 'mode_fare', 'sex_count'], axis=1)
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
print(f'score: {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')

score: 0.7038635542329971


In [6]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
print(f'score: {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')

score: 0.7834073637476873
